# 🏦 Minimal Colab: Fine-tune Small LLM for SG Financial Regulations

A streamlined notebook to run the improved pipeline only.


In [ ]:
# 1) Setup
!pip install -q torch transformers datasets peft accelerate bitsandbytes
!pip install -q nltk rouge-score pandas numpy

import nltk
nltk.download('punkt')
print('✅ Setup complete')


In [ ]:
# 2) Clone repo and check GPU
!git clone https://github.com/yihhan/finetune.git
%cd finetune

import torch
print('Device:', 'CUDA' if torch.cuda.is_available() else 'CPU')


In [ ]:
## 📊 Dataset Preparation

# 3) Enhanced dataset prep + inspection
import os, json, pandas as pd
qa = 'processed_data/enhanced_financial_regulation_qa.json'
tr = 'processed_data/enhanced_training_data.json'

if not (os.path.exists(qa) and os.path.exists(tr)):
    print("🚀 Generating enhanced dataset...")
    !python improved_dataset_prep.py
else:
    print('✅ Enhanced dataset exists, skipping generation')

# Show dataset details
with open(qa, 'r', encoding='utf-8') as f:
    data = json.load(f)
with open(tr, 'r', encoding='utf-8') as f:
    training_data = json.load(f)

print(f"\n📊 Dataset Summary:")
print(f"  Q&A pairs: {len(data)}")
print(f"  Training samples: {len(training_data)} (with augmentation)")
print(f"  Categories: {set(item['category'] for item in data)}")

print(f"\n📝 Sample Q&A:")
sample = data[0]
print(f"Q: {sample['question']}")
print(f"A: {sample['answer'][:200]}...")
print(f"Category: {sample['category']}")

# Category distribution
df = pd.DataFrame(data)
print(f"\n📈 Category distribution:")
print(df['category'].value_counts())


In [ ]:
## 🔥 AGGRESSIVE Flan-T5-BASE Training (FIXED!)

# 4) AGGRESSIVE training - previous was too conservative!
print("🔥 Starting AGGRESSIVE Flan-T5-BASE training!")
print("- Previous training was too conservative (identical responses)")
print("- NEW: LoRA r=32, alpha=64 (4x more aggressive)")
print("- NEW: Target 6 modules instead of 2 (q,v,k,o,wi,wo)")
print("- NEW: Learning rate 1e-4 (2x higher)")
print("- NEW: 4 epochs (more training)")
print("- Model output: flan_t5_base_fixed_model/")
print("- This should produce DIFFERENT responses from base model!")

!python flan_t5_base_train_fixed.py

print('✅ AGGRESSIVE training completed! Should show real improvement!')


In [ ]:
## 💬 Flan-T5-BASE Inference Demo

# 5) Test Flan-T5-BASE model - the working larger model!
print("🎯 Testing Flan-T5-BASE model (the one that actually works!):")
print("="*60)

# Quick test with the working base model first
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

print("Loading Flan-T5-BASE for quick test...")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

test_questions = [
    "What are capital requirements for banks?",
    "What is MAS in Singapore?",
    "Define financial regulation.",
]

for q in test_questions:
    inputs = tokenizer(f"Answer this question: {q}", return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50, num_beams=3)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Q: {q}")
    print(f"A: {response}")
    print("-" * 40)

print("✅ Flan-T5-BASE inference demo completed!")


In [ ]:
## 📈 AGGRESSIVE Model Evaluation & Comparison

# 6) Evaluate the AGGRESSIVE training results
print("📊 Running evaluation with AGGRESSIVE model...")
print("Comparing: Base Flan-T5-BASE vs AGGRESSIVE Fine-tuned model")

# Quick test first to see if responses are different
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import torch
from pathlib import Path

print("\n🧪 Quick comparison test:")

# Load base model
base_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Load aggressive model
try:
    lora_path = Path("flan_t5_base_fixed_model/lora_adapters")
    if lora_path.exists():
        print("Loading AGGRESSIVE LoRA adapters...")
        base_model_copy = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
        aggressive_model = PeftModel.from_pretrained(base_model_copy, lora_path)
        print("✅ AGGRESSIVE model loaded!")
    else:
        print("❌ AGGRESSIVE model not found")
        aggressive_model = base_model
except Exception as e:
    print(f"❌ Error loading AGGRESSIVE model: {e}")
    aggressive_model = base_model

# Test question
test_q = "What are the capital requirements for banks in Singapore?"
inputs = base_tokenizer(f"Answer this Singapore financial regulation question: {test_q}", return_tensors="pt")

# Base response
with torch.no_grad():
    base_out = base_model.generate(**inputs, max_new_tokens=50, num_beams=3)
base_response = base_tokenizer.decode(base_out[0], skip_special_tokens=True)

# Aggressive response  
with torch.no_grad():
    agg_out = aggressive_model.generate(**inputs, max_new_tokens=50, num_beams=3)
agg_response = base_tokenizer.decode(agg_out[0], skip_special_tokens=True)

print(f"\nQ: {test_q}")
print(f"Base Model: {base_response}")
print(f"AGGRESSIVE: {agg_response}")

if base_response != agg_response:
    print("✅ SUCCESS: Responses are DIFFERENT!")
    print("🎉 AGGRESSIVE training worked!")
else:
    print("❌ PROBLEM: Responses are still identical")
    print("😞 Need even more aggressive parameters")

print("\n" + "="*60)

# Summary of AGGRESSIVE training results
print("\n🔥 AGGRESSIVE TRAINING SUMMARY:")
print("="*60)
print("🎯 Goal: Fix identical responses from conservative training")
print("📊 Previous: r=8, alpha=16, 2 modules → identical responses")
print("🔥 AGGRESSIVE: r=32, alpha=64, 6 modules → should be different!")

print(f"\n🔍 What we learned:")
print(f"  ❌ DialoGPT: Wrong architecture (0.0001 BLEU)")
print(f"  ❌ Flan-T5-small: Too small/broken (gibberish)")
print(f"  ⚠️ Conservative LoRA: Too weak (identical responses)")
print(f"  🔥 AGGRESSIVE LoRA: Should finally work!")

print(f"\n📁 Model artifacts:")
print(f"  • flan_t5_base_fixed_model/ - AGGRESSIVE fine-tuned model")
print(f"  • Should produce DIFFERENT responses from base model")
print(f"  • Higher LoRA rank/alpha for actual learning")

if base_response != agg_response:
    print(f"\n🎉 BREAKTHROUGH: Fine-tuning finally works!")
    print(f"  • Base and fine-tuned responses are DIFFERENT")
    print(f"  • AGGRESSIVE parameters successful")
    print(f"  • Ready for production deployment!")
else:
    print(f"\n😞 Still need more work:")
    print(f"  • Responses still identical")
    print(f"  • May need even higher LoRA parameters")
    print(f"  • Or different training approach")


In [ ]:
## 🔥 AGGRESSIVE Training Pipeline Summary

# 7) AGGRESSIVE pipeline summary - fixing the identical response problem!
print("🔥 AGGRESSIVE TRAINING PIPELINE COMPLETED!")
print("="*50)
print("✅ Enhanced dataset: 21 Q&A pairs → 63 training samples")
print("✅ Found working base model: Flan-T5-BASE")
print("⚠️ Conservative training failed: Identical responses")
print("🔥 AGGRESSIVE training: Should fix the problem!")

print(f"\n📊 Training Evolution:")
print(f"  1️⃣ DialoGPT: Wrong architecture → 0.0001 BLEU")
print(f"  2️⃣ Flan-T5-small: Too small → gibberish responses")
print(f"  3️⃣ Flan-T5-base: Good base → coherent responses")
print(f"  4️⃣ Conservative LoRA: Too weak → identical responses")
print(f"  5️⃣ AGGRESSIVE LoRA: Strong params → should work!")

print(f"\n🔥 AGGRESSIVE Parameters:")
print(f"  • LoRA rank: 8 → 32 (4x higher)")
print(f"  • LoRA alpha: 16 → 64 (4x stronger)")
print(f"  • Target modules: 2 → 6 (3x more coverage)")
print(f"  • Learning rate: 5e-5 → 1e-4 (2x higher)")
print(f"  • Epochs: 2 → 4 (2x more training)")

print(f"\n📁 Generated artifacts:")
print(f"  • processed_data/enhanced_*.json - Training data")
print(f"  • flan_t5_base_fixed_model/ - AGGRESSIVE fine-tuned model")
print(f"  • Should produce DIFFERENT responses from base!")

print(f"\n🎯 Success Criteria:")
print(f"  ✅ Base model responses: Coherent but generic")
print(f"  🎯 Fine-tuned responses: Different + domain-specific")
print(f"  🎯 BLEU improvement: > 1.0x (not identical)")
print(f"  🎯 Singapore-specific: MAS, SGD, local regulations")

# Optional: Save to Google Drive
print(f"\n📦 Optional: Uncomment below to save to Google Drive")
print("# from google.colab import drive")
print("# drive.mount('/content/drive')")
print("# !cp -r flan_t5_base_fixed_model /content/drive/MyDrive/")

print(f"\n🔥 AGGRESSIVE TRAINING: The final attempt to make fine-tuning work!")
